In [2]:
#initialise

import extractor
extractor.main()
fps = extractor.fps
resolution = extractor.resolution





FPS: 30.0
Resolution: 720x720
Audio extracted to output_data/audio.mp3
Extracted 661 frames.
Feature data has been collected!
Writing data to csv
Features saved to output_data/output_features.csv
Extraction completed!


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  




In [4]:
# config:
unbiased = False # Should data be shuffled before training. Biased data is more accurate with reconstruction of the original video.
feature_threshold = 0.5 # between 0 and 1, how much should the data correlate to the target variable? 1 is 1 to 1 and 0 is not at all

In [5]:
import pydub

df = pd.read_csv('output_data/output_features.csv')
df['frame_index'] = df.index
df['normalized_index']=df['frame_index']/len(df)

audio_full = pydub.AudioSegment.from_mp3('output_data/audio.mp3')

slice_paths = []

for i in range(len(df['timestamp'])):
    frame_start = df['timestamp'][i]
    if i >= len(df['timestamp'])-1:
        frame_end = len(audio_full)
    else:
        frame_end = df['timestamp'][(i+1)]
    slice:pydub.AudioSegment = audio_full[frame_start:frame_end]
    slice_path = f'output_slices/audio_slice_{frame_start*0.001}_{i}.wav'
    slice.export(slice_path, format='wav')
    slice_paths.append(slice_path)

    




df['audio_slice_path'] = slice_paths
if unbiased:
    df = df.sample(frac=1.0,random_state=42).reset_index(drop=True)
print(df[['frame_filename','frame_index','normalized_index','audio_slice_path']].head())

                       frame_filename  frame_index  normalized_index  \
0    output_frames/frame_0_time_0.jpg            0          0.000000   
1   output_frames/frame_1_time_33.jpg            1          0.001513   
2   output_frames/frame_2_time_66.jpg            2          0.003026   
3  output_frames/frame_3_time_100.jpg            3          0.004539   
4  output_frames/frame_4_time_133.jpg            4          0.006051   

                        audio_slice_path  
0    output_slices/audio_slice_0.0_0.wav  
1  output_slices/audio_slice_0.033_1.wav  
2  output_slices/audio_slice_0.066_2.wav  
3    output_slices/audio_slice_0.1_3.wav  
4  output_slices/audio_slice_0.133_4.wav  


In [6]:
# Check shape
print("DataFrame shape:", df.shape)

# Check first few rows
print(df.head())



# Check if you accidentally only have one row
if df.shape[0] <= 1:
    print("Warning: Only 1 row detected!")

#df = df[df != 0] # remove redundant parameters
# Check for missing rows or NaNs
df = df.loc[:,(df != 0).any(axis=0)]
print(df.isnull().sum())



DataFrame shape: (661, 1517)
   timestamp                      frame_filename  hist_bin_0  hist_bin_1  \
0          0    output_frames/frame_0_time_0.jpg    0.999998    0.000072   
1         33   output_frames/frame_1_time_33.jpg    0.999785    0.008437   
2         66   output_frames/frame_2_time_66.jpg    0.992374    0.115958   
3        100  output_frames/frame_3_time_100.jpg    0.989822    0.133774   
4        133  output_frames/frame_4_time_133.jpg    0.988097    0.144113   

   hist_bin_2  hist_bin_3  hist_bin_4  hist_bin_5  hist_bin_6  hist_bin_7  \
0    0.000008    0.000000    0.000000    0.000000     0.00000    0.000000   
1    0.000189    0.000000    0.000000    0.000000     0.00000    0.000000   
2    0.001455    0.000282    0.000002    0.000000     0.00000    0.000000   
3    0.001915    0.000289    0.000002    0.000000     0.00000    0.000000   
4    0.002725    0.000428    0.000081    0.000063     0.00002    0.000056   

   ...  edge_993  edge_994  edge_995  edge_996  edg

In [7]:
#Correlation test?
float_df = df.select_dtypes(include=['float64'])

corr_matrix = float_df.corr()
normalized_index_corr = corr_matrix['normalized_index'].sort_values(ascending=False)
normalized_index_corr = normalized_index_corr.drop('normalized_index')
print(normalized_index_corr)

hist_bin_145    0.360664
hist_bin_211    0.357253
hist_bin_501    0.348983
hist_bin_291    0.346357
hist_bin_218    0.331281
                  ...   
hist_bin_78    -0.493177
hist_bin_150   -0.661907
hist_bin_143   -0.683861
hist_bin_142   -0.698862
hist_bin_141   -0.701348
Name: normalized_index, Length: 422, dtype: float64


In [8]:

X = df.drop(['timestamp','frame_filename','frame_index','normalized_index','audio_slice_path'], axis=1)
y = df['normalized_index']

In [9]:
# get strong features
strong_features = normalized_index_corr[abs(normalized_index_corr) > feature_threshold].index.to_list()
print(strong_features)
X = X[strong_features]

['hist_bin_150', 'hist_bin_143', 'hist_bin_142', 'hist_bin_141']


In [10]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

model.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [11]:
y_pred = model.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

MSE: 0.0309
R2 Score: 0.6530


In [13]:
df['predicted_normalized_index'] = model.predict(X)

# sort by predicted order
df_sorted = df.sort_values(by='predicted_normalized_index')

print(df_sorted[['frame_filename','predicted_normalized_index']].head(10))


                         frame_filename  predicted_normalized_index
10  output_frames/frame_10_time_333.jpg                    0.017761
9    output_frames/frame_9_time_300.jpg                    0.018003
8    output_frames/frame_8_time_266.jpg                    0.018094
11  output_frames/frame_11_time_366.jpg                    0.018835
13  output_frames/frame_13_time_433.jpg                    0.020862
12  output_frames/frame_12_time_400.jpg                    0.020893
14  output_frames/frame_14_time_466.jpg                    0.021195
15  output_frames/frame_15_time_500.jpg                    0.024009
16  output_frames/frame_16_time_533.jpg                    0.026309
18  output_frames/frame_18_time_600.jpg                    0.026536


In [14]:
import cv2

output_video = cv2.VideoWriter('output_data/reconstructed.avi',
                               cv2.VideoWriter_fourcc(*'XVID'),
                               fps,
                               resolution)  


for _, row in df_sorted.iterrows():
    frame = cv2.imread(row['frame_filename'])
    output_video.write(frame)

output_video.release()
print("Reconstructed Video Saved!!")

Reconstructed Video Saved!!


In [15]:
final_audio:pydub.AudioSegment = pydub.AudioSegment.empty()
for s in df['audio_slice_path']:
    sorted_slice = pydub.AudioSegment.from_wav(s)
    final_audio = final_audio.append(sorted_slice, crossfade=0)
final_audio.export('output_data/final_audio.mp3')

<_io.BufferedRandom name='output_data/final_audio.mp3'>

In [16]:
import subprocess

def combine_audio_video(reconstructed_video, audio_file, output_video):
    command = ['ffmpeg', '-i', reconstructed_video, '-i', audio_file, '-c:v', 'copy', '-c:a', 'aac', output_video, '-y']
    subprocess.run(command, check=True)
    print(f"Final video saved to {output_video}")

combine_audio_video('output_data/reconstructed.avi', 'output_data/final_audio.mp3','output_data/final_output.mp4')

Final video saved to output_data/final_output.mp4


In [17]:
print(df_sorted['audio_slice_path'])
df.to_csv('output_data/final_output_data.csv')

10                output_slices/audio_slice_0.333_10.wav
9                    output_slices/audio_slice_0.3_9.wav
8                  output_slices/audio_slice_0.266_8.wav
11                output_slices/audio_slice_0.366_11.wav
13                output_slices/audio_slice_0.433_13.wav
                             ...                        
627    output_slices/audio_slice_20.900000000000002_6...
628             output_slices/audio_slice_20.933_628.wav
619             output_slices/audio_slice_20.633_619.wav
621               output_slices/audio_slice_20.7_621.wav
622             output_slices/audio_slice_20.733_622.wav
Name: audio_slice_path, Length: 661, dtype: object
